<a href="https://colab.research.google.com/github/Nitin-Dwivedi-7/HSI-/blob/master/HSI_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import numpy as np                                       #importing all the libs
import scipy as s
from scipy.io import loadmat
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint, TensorBoard


import pylab as plt
import pandas as pd
import tensorflow as  tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten, Dropout, LSTM
import tensorflow.keras.layers as kl
from tensorflow.keras.layers import BatchNormalization 
from tensorflow.keras import regularizers

Using TensorFlow backend.


In [0]:
data1= loadmat("/content/drive/My Drive/data sets/Salinas_corrected.mat")          #import
image=data1["salinas_corrected"] 

data2=loadmat("/content/drive/My Drive/data sets/Salinas_gt.mat")                                #import
ground_truth=data2["salinas_gt"]               #storing array from dict to a var



In [18]:
image_with_gt = np.dstack((image, ground_truth))
final_output = image_with_gt.reshape(ground_truth.size, image.shape[2]+1)
data=pd.DataFrame(final_output)
data.shape

(111104, 205)

In [19]:
data = data[data.iloc[:, -1] != 0]
data.shape

(54129, 205)

In [0]:
gt=data.loc[:,data.shape[1]-1]
dataset=data.loc[:, 0:data.shape[1]-2]

In [21]:
from sklearn import preprocessing
dataset = preprocessing.scale(dataset)
pd.DataFrame(dataset).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203
0,0.139160,1.146405,1.193081,1.718903,1.718909,1.601013,1.655999,1.610411,1.639688,1.570228,1.685770,1.633315,1.673390,1.682833,1.784031,1.814051,1.879123,1.896434,1.902166,1.889257,1.869346,1.816878,1.803749,1.808617,1.796376,1.762614,1.760216,1.742127,1.734016,1.685356,1.638786,1.644449,1.655603,1.642578,1.675813,1.912470,2.182915,2.548387,1.987607,1.008974,...,0.350240,0.334835,0.345350,0.333740,0.350236,0.363033,0.376761,0.385612,0.428009,0.432381,0.522402,0.477160,0.422721,0.410690,0.372893,0.341626,0.288130,0.289784,0.236574,0.272978,0.267500,0.242950,0.295106,0.269246,0.239269,0.218383,0.314710,0.253831,0.253026,0.270428,0.272453,0.213020,0.253812,0.109516,0.114345,0.102462,0.166668,0.048395,-0.601151,-0.186720
1,1.249739,0.139285,1.193081,1.464188,1.455744,1.601013,1.581776,1.561417,1.529830,1.599557,1.634614,1.589513,1.673390,1.664461,1.697442,1.714262,1.769551,1.802702,1.826031,1.833292,1.779827,1.743497,1.719623,1.707785,1.693152,1.712444,1.707819,1.682525,1.639003,1.601144,1.574578,1.562422,1.570655,1.518795,1.620714,1.816487,2.120580,2.385546,1.908261,0.923938,...,0.332267,0.348637,0.345350,0.347609,0.360123,0.347218,0.360416,0.368804,0.428009,0.488530,0.455978,0.459355,0.422721,0.410690,0.372893,0.341626,0.295089,0.254094,0.259282,0.280903,0.229593,0.207981,0.311495,0.250542,0.270991,0.382466,0.326382,0.253831,0.328336,0.294208,0.242455,0.280635,0.169295,0.175487,0.273579,0.175003,0.062968,0.139396,0.066622,0.645191
2,1.249739,1.146405,1.193081,1.464188,1.455744,1.515566,1.511264,1.561417,1.567281,1.599557,1.585589,1.611414,1.612287,1.664461,1.679771,1.730591,1.769551,1.787080,1.795578,1.792142,1.779827,1.743497,1.719623,1.718686,1.714058,1.681570,1.656700,1.672591,1.629258,1.601144,1.565405,1.553308,1.552036,1.527964,1.565615,1.767162,2.077539,2.372739,1.842801,0.869610,...,0.363720,0.357838,0.331679,0.379968,0.330462,0.384120,0.349519,0.419230,0.387583,0.413664,0.468055,0.405940,0.428748,0.410690,0.392388,0.341626,0.295089,0.282646,0.297129,0.272978,0.267500,0.391568,0.254134,0.316005,0.260417,0.276295,0.291364,0.253831,0.278129,0.234758,0.197458,0.246827,0.169295,0.142502,0.167423,0.102462,0.270368,-0.133606,0.066622,-0.035463
3,1.249739,1.146405,1.193081,1.464188,1.585096,1.515566,1.581776,1.610411,1.674642,1.687544,1.736926,1.722910,1.799538,1.776528,1.819373,1.899324,1.958651,1.990167,1.991835,1.986373,1.985564,1.928419,1.897690,1.887648,1.878695,1.862953,1.862454,1.842705,1.791268,1.751260,1.692675,1.662678,1.694004,1.687278,1.730911,1.961794,2.361015,2.749650,2.199856,1.124716,...,0.350240,0.334835,0.354465,0.347609,0.340349,0.389392,0.376761,0.385612,0.451109,0.469814,0.486171,0.489030,0.422721,0.429439,0.379391,0.321182,0.246375,0.289784,0.259282,0.241280,0.237175,0.146785,0.278717,0.250542,0.270991,0.218383,0.244674,0.253831,0.215370,0.234758,0.242455,0.111597,0.197467,0.274442,0.061267,0.138733,-0.040732,0.139396,-0.378560,-0.337977
4,1.249739,0.139285,1.193081,1.464188,1.585096,1.690345,1.797024,1.812511,1.784500,1.804859,1.837108,1.880200,1.902033,1.943711,1.941304,2.015441,2.084129,2.146387,2.157639,2.152622,2.100212,2.051701,2.016869,2.010282,1.971466,1.964579,1.976193,1.932108,1.876536,1.845235,1.766057,1.781162,1.797571,1.749170,1.822743,2.077773,2.487170,2.950914,2.398220,1.279434,...,0.341253,0.367039,0.336236,0.356854,0.394726,0.363033,0.398554,0.424833,0.428009,0.488530,0.486171,0.471225,0.440801,0.416939,0.346901,0.341626,0.309008,0.289784,0.312267,0.257129,0.267500,0.312888,0.311495,0.250542,0.218121,0.237687,0.314710,0.303830,0.278129,0.317988,0.242455,0.246827,0.141123,0.109516,0.008189,0.066192,0.062968,-0.133606,-0.155969,-0.035463


In [22]:
from sklearn.decomposition import PCA 
pca = PCA(n_components = 100) 
dataset_p = pca.fit_transform(dataset) 
 
print(dataset_p.shape) 
pd.DataFrame(dataset_p).head()

(54129, 100)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,7.370062,14.608778,-4.599675,-1.201339,0.508440,0.727780,-0.549821,0.445545,0.028141,0.054854,-0.327387,0.073855,0.074987,0.108915,0.074997,0.216134,0.009959,-0.124753,-0.069380,-0.003202,0.019001,0.066638,0.169490,-0.029001,-0.154714,0.007817,-0.037724,-0.087384,0.007621,0.014667,-0.015506,-0.035834,0.073077,-0.026413,0.010200,-0.055686,-0.030678,0.022374,-0.033052,0.029197,...,0.029935,0.010841,-0.050130,-0.033723,-0.013038,-0.010320,-0.013169,0.022019,-0.001617,-0.006271,0.051121,0.004316,-0.011426,-0.005984,-0.009281,-0.028474,0.054789,-0.018959,-0.006114,0.006247,-0.011573,0.001954,-0.071041,0.004523,-0.040480,0.021778,-0.008758,0.009027,0.019310,-0.008294,0.006203,0.032576,-0.009595,-0.001622,-0.001182,0.012319,-0.014001,0.028950,0.006347,-0.033009
1,7.363425,13.659417,-4.158736,-1.199417,0.128856,0.417383,0.563691,-0.800512,0.310611,0.035619,-0.162785,0.146928,0.408660,-0.156829,0.231279,-0.073694,-0.116797,-0.066257,-0.180632,-0.219458,0.047397,-0.041414,-0.005372,0.078568,0.036102,0.077788,-0.040720,-0.050776,0.057357,-0.041697,-0.014953,-0.005061,0.047308,0.019049,0.051438,-0.017145,-0.071886,-0.023056,0.094803,-0.091152,...,-0.071544,-0.009774,0.032046,0.069470,-0.009734,0.011368,-0.010018,-0.008659,0.023108,-0.025151,-0.004723,0.000693,0.006727,-0.033559,0.021200,-0.014461,0.013773,-0.014596,-0.017626,-0.032967,-0.014500,0.011706,-0.027562,0.015684,0.009364,-0.025798,-0.002908,0.016902,-0.001863,-0.010116,0.012088,-0.053327,-0.014367,-0.014286,0.019103,-0.009678,0.024044,0.002444,-0.002899,-0.003123
2,7.378918,13.431269,-4.128623,-1.332910,0.460753,-0.144338,0.166293,-0.127029,-0.032093,0.188198,-0.108607,-0.207122,0.011919,0.145396,0.273241,-0.113422,-0.160272,0.061775,-0.113732,0.067730,0.103780,0.096573,0.108757,-0.106323,-0.000625,0.044152,-0.085867,-0.023375,0.022191,-0.054967,0.081102,-0.025286,0.007353,-0.064981,-0.074859,0.087479,-0.000069,-0.045656,0.033738,-0.004396,...,-0.033172,-0.046561,-0.030147,0.001352,-0.029006,0.048868,0.004410,-0.031606,-0.022789,-0.004262,-0.028431,-0.023196,-0.001198,0.029095,-0.000983,0.007683,0.010973,0.022860,0.009905,0.023139,-0.008437,-0.018092,-0.016339,0.001976,-0.010868,0.008428,-0.000695,0.001191,0.024649,0.017074,-0.002930,0.042004,-0.011932,-0.037185,-0.009881,-0.042222,0.016626,-0.002005,-0.026463,-0.013925
3,7.388459,15.706656,-4.729384,-1.142663,0.738661,0.093819,-0.193816,-0.281561,-0.353200,0.137065,-0.143740,-0.161879,0.014176,0.052323,-0.257468,0.123366,0.154181,-0.121835,-0.066213,0.008408,0.088289,-0.061703,-0.010432,0.051074,-0.121462,-0.051563,0.098329,0.046788,-0.038788,-0.119787,0.031492,0.008300,0.088558,0.095355,0.002579,-0.036044,0.092392,0.062283,-0.014961,0.023981,...,0.049690,0.020483,0.009667,0.013924,0.021938,0.010823,0.023360,0.014535,-0.015102,-0.040554,-0.010959,-0.063052,-0.002630,-0.006850,-0.028168,-0.025428,0.018697,0.015921,-0.016032,-0.026242,-0.014396,-0.005970,-0.008323,0.000229,-0.019372,-0.009329,-0.042429,-0.004273,0.022106,-0.007014,0.020096,0.004833,-0.006329,0.027887,0.012431,0.021093,-0.007494,0.029382,-0.003052,-0.036639
4,7.586524,16.732894,-4.923359,-1.209449,0.577338,0.844137,0.099221,-1.021283,-0.108163,0.041838,-0.059513,-0.095833,0.067109,-0.202952,-0.217730,-0.081019,-0.084437,0.045605,-0.130969,0.012515,0.049561,-0.068505,0.015813,-0.091131,0.010833,-0.034429,-0.035918,-0.124572,-0.005294,-0.052729,0.091344,-0.003252,-0.018626,0.018725,0.023572,0.039334,-0.062479,-0.038971,-0.003937,-0.032545,...,-0.008775,0.008855,-0.066150,0.039867,0.024503,-0.004376,0.030750,0.003393,0.021048,-0.064050,-0.047220,0.004375,-0.028792,-0.024067,-0.014008,-0.015787,-0.016598,0.003919,0.012628,0.008742,0.001596,-0.015871,0.012583,-0.014328,-0.047793,0.008325,0.010000,0.017791,-0.007704,0.020471,0.012906,-0.

In [81]:
from sklearn.preprocessing import OneHotEncoder 

onehotencoder = OneHotEncoder() 
gt = onehotencoder.fit_transform(np.array(gt).reshape(-1,1)).toarray() 
print(gt)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [82]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(dataset_p,gt, test_size=0.30, random_state=42)
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)


(37890, 100) (37890, 16) (16239, 100) (16239, 16)


In [87]:
first_layer_size = 32                  #############################nodes in first layer
model = Sequential()
model.add(Dense(first_layer_size, activation='sigmoid', input_shape=(100,)))#####################adding layers
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(16, activation='softmax' ))

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_37 (Dense)             (None, 32)                3232      
_________________________________________________________________
dense_38 (Dense)             (None, 100)               3300      
_________________________________________________________________
dense_39 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_40 (Dense)             (None, 16)                1616      
Total params: 18,248
Trainable params: 18,248
Non-trainable params: 0
_________________________________________________________________


In [0]:
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [89]:
print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

(37890, 100)
(37890, 16)
(16239, 100)
(16239, 16)


In [91]:

history = model.fit(train_X,train_y,
                    batch_size=1000,
                    epochs=500,
                    verbose=1)


Epoch 1/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1322 - accuracy: 0.9474
Epoch 2/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1317 - accuracy: 0.9478
Epoch 3/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1304 - accuracy: 0.9483
Epoch 4/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1304 - accuracy: 0.9478
Epoch 5/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1292 - accuracy: 0.9489
Epoch 6/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1296 - accuracy: 0.9479
Epoch 7/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1277 - accuracy: 0.9494
Epoch 8/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1266 - accuracy: 0.9495
Epoch 9/500
37890/37890 [==============================] - 0s 6us/step - loss: 0.1260 - accuracy: 0.9497
Epoch 10/500
37890/37890 [=============================

In [95]:
score = model.evaluate(test_X, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Test loss: 0.109007554094739
Test accuracy: 0.962189793586731


In [101]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
prediction = model.predict(test_X)
pd.DataFrame(prediction)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1.244482e-12,1.000000e+00,4.726914e-22,9.754703e-27,4.894792e-17,3.822111e-08,3.125364e-12,2.855993e-11,1.671377e-14,1.347583e-09,2.167864e-16,1.965225e-35,3.659335e-26,1.910499e-17,1.026120e-27,8.288517e-13
1,1.938571e-26,1.126437e-24,8.690717e-09,5.061985e-09,1.000000e+00,6.373604e-15,2.668581e-14,1.280132e-16,1.519948e-19,6.624897e-11,6.380650e-18,1.452664e-13,6.377230e-22,4.959289e-20,6.003495e-10,2.976902e-19
2,2.161118e-28,9.803005e-18,6.238842e-10,3.679741e-17,1.174744e-06,1.452405e-12,4.508072e-22,2.338543e-11,9.999986e-01,2.237949e-07,1.103113e-09,4.060447e-15,1.119649e-26,1.200948e-16,7.735239e-15,2.697827e-20
3,1.000000e+00,1.035972e-10,1.260705e-29,4.965926e-23,4.074240e-18,2.488458e-11,9.900927e-09,6.987590e-11,2.124475e-23,3.000973e-14,8.228265e-21,9.349412e-30,1.222379e-21,9.037728e-16,3.186347e-21,2.330023e-08
4,3.555527e-15,1.616428e-23,1.542617e-17,1.507767e-11,9.338738e-11,2.166251e-13,2.050041e-14,1.549311e-03,2.788707e-18,3.480914e-09,1.843673e-18,2.311706e-08,6.009204e-13,4.533107e-08,9.984506e-01,2.316081e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16234,5.396915e-11,4.435988e-22,4.890414e-22,2.691541e-12,3.789228e-13,4.025522e-15,3.501105e-15,9.869145e-01,8.954690e-22,4.116193e-11,1.695765e-22,1.750482e-11,8.706206e-11,4.082925e-10,1.308552e-02,4.537613e-09
16235,5.486642e-11,2.007704e-21,5.337454e-20,3.852650e-10,5.026926e-12,5.193558e-15,2.572838e-14,9.319389e-01,1.050454e-20,5.162559e-10,2.803245e-21,4.781813e-10,1.412405e-09,3.536699e-10,6.806104e-02,4.829841e-08
16236,7.184018e-12,6.303631e-22,1.052961e-20,8.110492e-12,9.037799e-12,4.265089e-14,2.442465e-14,9.081187e-02,6.078425e-21,6.680148e-10,3.824675e-21,1.483755e-10,6.678273e-11,8.290116e-09,9.091882e-01,1.781244e-09
16237,9.236895e-11,5.281238e-21,7.749557e-23,2.988555e-14,8.435507e-12,1.672329e-12,9.461231e-14,8.809615e-01,2.204985e-21,1.332689e-09,1.733978e-20,1.332198e-12,8.374113e-11,1.060656e-07,1.190383e-01,1.113089e-08
